In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("chat_completion", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))

In [2]:
kernel

In [3]:
sk_prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

In [4]:
chat_function = kernel.create_semantic_function(
    prompt_template = sk_prompt, 
    function_name = "ChatBot", 
    skill_name="ChatBotSkill",
    max_tokens=2000, 
    temperature=0.7, 
    top_p=0.5)

In [5]:
context = kernel.create_new_context()
context["history"] = """
The men's high jump event at the 2020 Summer Olympics took place between 30 July and 1 August 2021 at the Olympic Stadium.
33 athletes from 24 nations competed; the total possible number depended on how many nations would use universality places
to enter athletes in addition to the 32 qualifying through mark or ranking (no universality places were used in 2021).
Italian athlete Gianmarco Tamberi along with Qatari athlete Mutaz Essa Barshim emerged as joint winners of the event following
a tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal in a rare instance
where the athletes of different nations had agreed to share the same medal in the history of Olympics.
Barshim in particular was heard to ask a competition official "Can we have two golds?" in response to being offered a
'jump off'. Maksim Nedasekau of Belarus took bronze. The medals were the first ever in the men's high jump for Italy and
Belarus, the first gold in the men's high jump for Italy and Qatar, and the third consecutive medal in the men's high jump
for Qatar (all by Barshim). Barshim became only the second man to earn three medals in high jump, joining Patrik Sjöberg
of Sweden (1984 to 1992)."""

In [6]:
context["user_input"] = "Who won the gold in 2020 Summer Olympics men's high jump?"
bot_answer = await chat_function.invoke_async(context=context)
print(bot_answer)

In the 2020 Summer Olympics men's high jump, Italian athlete Gianmarco Tamberi and Qatari athlete Mutaz Essa Barshim both won gold medals. They agreed to share the gold after tying with a clearance of 2.37m.


In [7]:
context["history"] += f"\nUser: {context['user_input']}\nChatBot: {bot_answer}\n"
print(context["history"])


The men's high jump event at the 2020 Summer Olympics took place between 30 July and 1 August 2021 at the Olympic Stadium.
33 athletes from 24 nations competed; the total possible number depended on how many nations would use universality places
to enter athletes in addition to the 32 qualifying through mark or ranking (no universality places were used in 2021).
Italian athlete Gianmarco Tamberi along with Qatari athlete Mutaz Essa Barshim emerged as joint winners of the event following
a tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal in a rare instance
where the athletes of different nations had agreed to share the same medal in the history of Olympics.
Barshim in particular was heard to ask a competition official "Can we have two golds?" in response to being offered a
'jump off'. Maksim Nedasekau of Belarus took bronze. The medals were the first ever in the men's high jump for Italy and
Belarus, the first gold in the men's hig

In [8]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    context["user_input"] = input_text

    # Process the user message and get an answer
    answer = await chat_function.invoke_async(context=context)

    # Show the response
    print(f"ChatBot: {answer}")

    # Append the new interaction to the chat history
    context["history"] += f"\nUser: {input_text}\nChatBot: {answer}\n"

In [9]:
await chat("Who won the bronze in 2020 Summer Olympics men's high jump?")

User: Who won the bronze in 2020 Summer Olympics men's high jump?
ChatBot: Maksim Nedasekau of Belarus won the bronze medal in the 2020 Summer Olympics men's high jump.


In [10]:
await chat("Who won the silver")

User: Who won the silver
ChatBot: In the 2020 Summer Olympics men's high jump, there was no silver medal awarded because the gold medal was shared between Gianmarco Tamberi of Italy and Mutaz Essa Barshim of Qatar. Maksim Nedasekau of Belarus won the bronze medal.


In [11]:
await chat("Who won the bronze")

User: Who won the bronze
ChatBot: Maksim Nedasekau of Belarus won the bronze medal in the 2020 Summer Olympics men's high jump.


In [12]:
print(context["history"])


The men's high jump event at the 2020 Summer Olympics took place between 30 July and 1 August 2021 at the Olympic Stadium.
33 athletes from 24 nations competed; the total possible number depended on how many nations would use universality places
to enter athletes in addition to the 32 qualifying through mark or ranking (no universality places were used in 2021).
Italian athlete Gianmarco Tamberi along with Qatari athlete Mutaz Essa Barshim emerged as joint winners of the event following
a tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal in a rare instance
where the athletes of different nations had agreed to share the same medal in the history of Olympics.
Barshim in particular was heard to ask a competition official "Can we have two golds?" in response to being offered a
'jump off'. Maksim Nedasekau of Belarus took bronze. The medals were the first ever in the men's high jump for Italy and
Belarus, the first gold in the men's hig

In [13]:
await chat("How was the Olympic Stadium?")

User: How was the Olympic Stadium?
ChatBot: The Olympic Stadium for the 2020 Summer Olympics, also known as the Japan National Stadium, is a multi-purpose stadium located in Tokyo, Japan. It was designed by architect Kengo Kuma and completed in November 2019. The stadium has a seating capacity of 68,000 and features a modern design with a focus on sustainability and harmony with the surrounding environment. It was the main venue for the opening and closing ceremonies, as well as hosting athletics events and football matches during the Olympics. Overall, the Olympic Stadium received positive reviews for its design, functionality, and aesthetics.
